## Liens à explorer :

https://www.programiz.com/python-programming/methods/built-in/super
https://stackoverflow.com/questions/222877/what-does-super-do-in-python
https://stackoverflow.com/questions/576169/understanding-python-super-with-init-methods?noredirect=1&lq=1
https://www.artima.com/weblogs/viewpost.jsp?thread=236275
https://www.artima.com/weblogs/viewpost.jsp?thread=236278
https://stackoverflow.com/questions/2771904/which-of-the-4-ways-to-call-super-in-python-3-to-use
http://www.cs.utexas.edu/~cannata/cs345/Class%20Notes/15%20python_attributes_and_methods.pdf
https://rushter.com/blog/python-class-internals/
https://www.protechtraining.com/bookshelf/python_fundamentals_tutorial/oop

Pour la partie sur les mixins : https://towardsdatascience.com/how-to-connect-objects-with-each-other-in-different-situations-with-pythonic-ways-d3aaf4c89553

In [15]:
class A(object): 
    def some_method(self):
        print 'Calling A.some_method'
        
class B(A):
    def some_method(self):
        print 'Calling B.some_method'
        super(B, self).some_method()

b = B()
b.some_method()

Calling B.some_method
Calling A.some_method


##### En Python 2, super doit prendre au moins un argument de type ```type``` (un objet class). Change en Python 3  

### super(type)
https://stackoverflow.com/questions/30190185/how-can-i-use-super-with-one-argument-in-python

In [30]:
B.__mro__

(__main__.B, __main__.A, object)

In [31]:
super(B).some_method

AttributeError: 'super' object has no attribute 'some_method'

### super(type, obj) avec obj instance de type (isinstance(obj, type) évalue à True)

In [10]:
super(B, b).some_method

<bound method B.some_method of <__main__.B object at 0x7fe614a17c10>>

In [17]:
m = super(B, b).some_method
m()

Calling A.some_method


### super(type, type2) avec type2 sous-classe de type (issubclass(type2, type) évalue à True)

In [11]:
super(B, B).some_method

<unbound method B.some_method>

In [18]:
m = super(B, B).some_method
m()

TypeError: unbound method some_method() must be called with B instance as first argument (got nothing instead)

In [19]:
m = super(B, B).some_method
m(b)

Calling A.some_method


### Autres

In [5]:
class C(object):
    x = 1
    def __init__(self, y):
        self.prop = y
        
    def f(self):
        pass
        
c1 = C(1)
c2 = C(2)

In [6]:
C.__dict__['f']

<function __main__.C.f(self)>

In [7]:
C.f

<function __main__.C.f(self)>

In [8]:
c1.f

<bound method C.f of <__main__.C object at 0x7f8d5c29d710>>

### Tests bis 

In [6]:
class B(object):
    pass

class C(B):
    pass

class D(C):
    pass

d = D()

# instance-bound syntax (instance-bound super object)
# If asked for instance-methods : returns bound-methods (bound to __self__)
# If asked for class-methods : returns bound-methods (bound to __self_class__)
# If asked for static-methods : returns functions (known as unbound methods in Python 2)
bsup = super(C, d)
assert bsup.__thisclass__ is C
assert bsup.__self__ is d
assert bsup.__self_class__ is D

# class-bound syntax (class-bound super object)
# If asked for instance methods : returns functions (known as unbound methods in Python 2)
# If asked for class methods : returns bound-methods (bound to __self__)
# If asked for static-methods : returns functions (known as unbound methods in Python 2)
Bsup = super(C, D)
assert Bsup.__thisclass__ is C
assert Bsup.__self__ is D
assert Bsup.__self_class__ is D

# unbound syntax (unbound super object)
usup = super(C)
assert usup.__thisclass__ is C
assert usup.__self__ is None
assert usup.__self_class__ is None

Super object possède trois attributs spécifiques : 
* __thisclass__ : type object correspondant au premier argument
* __self__ : objet correspondant au second argument (None si absent)
* __self_class__ : type(obj) si le second argument est une instance ou type object correspondant au second argument si celui-ci est un type (None si absent). 

Forme unbound : 
Peu recommandée. Peu d'usages, dépréciation future possible. "Does not dispatch attribute access to class" : requiert d'en passer par le protocole descriptor pour être utilisé.
Cf. proxy design pattern pour mieux comprendre ?

In [54]:
class A(object):
    a = 0
    def some_method(self):
        print('Instance method A.some_method')
        
    def print_name(self):
        print(self.name)

class B(A):
    def some_method(self):
        print('Instance method B.some_method')


class C(A):
    a = 1
    
    @classmethod
    def some_cmethod(cls):
        print('I inherit from C')
        
    @staticmethod
    def some_smethod():
        print('C.some_smethod')

class D(B, C):
    def __init__(self, name):
        self.name = name
        
    def some_method(self):
        print('Instance method D.some_method')


d = D("My name is d")
e = D("My name is e")

print(D.__mro__)
print(super(D, d).a)
print(B.__mro__)
print(super(B, d).a)

(<class '__main__.D'>, <class '__main__.B'>, <class '__main__.C'>, <class '__main__.A'>, <class 'object'>)
1
(<class '__main__.B'>, <class '__main__.A'>, <class 'object'>)
1


bound-method = fonction (partielle) dont le premier argument est fixé égal à __self__ de l'objet super si méthode d'instance ou __self_class__ de super su méthode de classe.
unbound-method = retourne la fonction, sans arguments fixés, quelle qu'en soit sa nature (pas possible de se voir retourné une class method unbound (?)).

In [17]:
super(D, d).some_method

<bound method B.some_method of <__main__.D object at 0x7fb006e43c50>>

In [18]:
super(C, d).some_method

<bound method A.some_method of <__main__.D object at 0x7fb006e43c50>>

In [19]:
super(D, d).some_method()

B.some_method


In [20]:
super(C, d).some_method()

A.some_method


In [24]:
super(D, d).print_name

<bound method A.print_name of <__main__.D object at 0x7fb006e4a860>>

In [25]:
super(D, d).print_name()

My name is d


In [29]:
super(D, D).print_name

<function __main__.A.print_name(self)>

In [32]:
super(D, D).print_name(d)

My name is d


In [31]:
super(D, D).print_name(e)

My name is e


Sur les class methods 

In [49]:
super(D, d).some_cmethod # still bounded => on lui passera __self_class__

<bound method C.some_cmethod of <class '__main__.D'>>

In [50]:
super(D, d).some_cmethod()

I inherit from C


In [41]:
super(D, D).some_cmethod

<bound method C.some_cmethod of <class '__main__.D'>>

In [42]:
super(D, D).some_cmethod()

I inherit from C


Sur les static methods 

In [44]:
super(D, d).some_smethod

<function __main__.C.some_smethod()>

In [45]:
super(D, d).some_smethod()

C.some_smethod


In [47]:
super(D, D).some_smethod

<function __main__.C.some_smethod()>

In [48]:
super(D, D).some_smethod()

C.some_smethod


Sur les méthodes d'instance

In [51]:
super(D, d).some_method # La repr dit à quel objet la bound-method est liée

<bound method B.some_method of <__main__.D object at 0x7fb006e5ce48>>

In [55]:
super(D, d).some_method()

Instance method B.some_method


In [56]:
super(D, D).some_method

<function __main__.B.some_method(self)>

In [58]:
super(D, D).some_method(d)

Instance method B.some_method


#### super sans arguments dans Python 3 : 
super sans argument correspond à super(type(self), self) où self est l'objet courant. Ne peut évidemment s'employer qu'à l'intérieur d'une méthode d'instance.

In [80]:
class A(object):
    def some_method(self):
        print('Instance method A.some_method')
    
class B(A):
    def show_super(self):
        sup = super()
        print(sup.__thisclass__)
        print(self)
        print(sup.__self__)
        print(sup.__self_class__) 
        # Faire ça plus classe avec des is ou assert
        # super() is super(B, self)
        
b = B()
b.show_super()

<class '__main__.B'>
<class '__main__.B'>


#### Sur les arguments de super 
Le premier : forcément un type, sa valeur est ensuite assignée à l'argument __thisclass__ de l'objet super. Il donne la classe dont le __mro__ servira à la résolution du scope, c'est le point de départ de la recherche dans la lignée linéarisée.  
Le second : si présent, sa valeur est ensuite assignée à l'argument __self__ et c'est à cette valeur que sera liée la fonction si cela à un sens, l'ensemble formant alors une bound method. Ne peut être qu'une instance du premier argument (si object) ou une sous-classe de celui-ci (si type) (si l'argument ne vérifiait pas ces conditions, il y aurait de fortes chances que les opérations demandées n'aient aucun sens et génèrent des erreurs).

Donné pour la compréhension, en pratique la configuration suivante n'a que peu de cas d'usages en plus d'impliquer une syntaxe assez obscure. On peut le voir comme une illustration du concept de descriptor en Python (https://stackoverflow.com/questions/30190185/how-can-i-use-super-with-one-argument-in-python) Dépréciation envisagée en Python 3 mais finalement non réalisée.
En absence de second argument, l'objet super sera unbound (__self__ = None). Se rappeler alors que super est un descriptor : ses attributs ne sont pas les objets qu'on recherche : super(A).some_method va renvoyer une AttributeError : l'objet super ne possède pas d'objet some_method, il faut l'utiliser (et donc utiliser le protocol descriptor) de façon à ce qu'il retourne ce que l'on veut. La syntaxe qu'on est alors contraint d'utiliser n'est qu'une explicitation de ce que fait Python quand on passe deux arguments à super (?).
Remarque le __get__ retourne un nouvel objet super mais bounded.

In [67]:
super(B) # unbounded, se voit dans la repr qui s'écrit <super: self.__thisclass__, self.__self__> 

<super: __main__.B, None>

In [71]:
sup = super(B)
sup.__self__ is None

True

In [72]:
super(B).__get__(b) # bounded

<super: __main__.B, <__main__.B at 0x7fb006e60ba8>>

In [76]:
sup = super(B).__get__(b) # 2e argument de __get__ facultatif 
sup.__self__

In [77]:
super(B).__get__(b).some_method

<bound method A.some_method of <__main__.B object at 0x7fb006e60ba8>>

In [81]:
super(B).__get__(b).some_method()

Instance method A.some_method
